In [1]:
#! pip install pyarrow matplotlib sentencepiece pandas
import torch
import os
import sys
import numpy as np
import math
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerDecoder, TransformerDecoderLayer
import sentencepiece as spm
import matplotlib.pyplot as plt
import multiprocessing
import time
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

if torch.backends.mps.is_available():  # Check for Apple Silicon GPU availability (requires PyTorch 1.12 or later)
    device = torch.device("mps")
elif torch.cuda.is_available():  # Check for NVIDIA GPU availability
    device = torch.device("cuda")
    torch.cuda.empty_cache()
else:
    device = torch.device("cpu")  # Fall back to CPU

print(f"Using device: {device}")

Using device: mps


Dataset Import

In [16]:
# Read data from both Parquet files
train_0 = pd.read_parquet('../dataset/0000.parquet')
train_1 = pd.read_parquet('../dataset/0001.parquet')
train_2 = pd.read_parquet('../dataset/0000.parquet')
train_3 = pd.read_parquet('../dataset/0001.parquet')
train_4 = pd.read_parquet('../dataset/0000.parquet')
train_5 = pd.read_parquet('../dataset/0001.parquet')
train_6 = pd.read_parquet('../dataset/0000.parquet')
train_7 = pd.read_parquet('../dataset/0001.parquet')
train_8 = pd.read_parquet('../dataset/0001.parquet')

train = pd.concat([train_0, train_1, train_2, train_3, train_4, train_5, train_6, train_7, train_8,], ignore_index=True)
train = train.reset_index(drop=True)

# # Concatenate elements within each row
# train['caption'] = train['caption'].apply('\n'.join)
# # Concatenate all rows into one long text string
# long_text = train['caption'].str.cat(sep='\n')
# with open('../dataset/corpus.txt', 'w') as file:
#     file.write(long_text)

# Train SentencePiece model directly from Python list of sentences
spm.SentencePieceTrainer.train(input='../dataset/corpus.txt', model_prefix='spm_full_text_model', vocab_size=10000, model_type='unigram')

print(train.shape)
print(train['caption'])
train = train.iloc[:100]

(33700, 6)
0        [Two young guys with shaggy hair look at their...
1        [Several men in hard hats are operating a gian...
2        [A child in a pink dress is climbing up a set ...
3        [Someone in a blue shirt and hat is standing o...
4        [Two men, one in a gray shirt, one in a black ...
                               ...                        
33695    [A baby is hold an adults fingers on the beach...
33696    [Two men wearing hats play accordions while st...
33697    [A boy with whipped cream on his face after sm...
33698    [A boy in a red and white shirt kicks a large ...
33699    [One person holds a brown and white puppy whil...
Name: caption, Length: 33700, dtype: object


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../dataset/corpus.txt
  input_format: 
  model_prefix: spm_full_text_model
  model_type: UNIGRAM
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differenti

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load('spm_05_text_model.model')